In [1]:
import pandas as pd
df_servers = pd.read_csv('./data/servers.csv')
df_servers

,server_generation,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee
0,CPU.S1,CPU,"[1,60]",15000,2,400,60,96,1000,288
1,CPU.S2,CPU,"[37,96]",16000,2,460,75,96,1000,308
2,CPU.S3,CPU,"[73,132]",19500,2,800,120,96,1000,375
3,CPU.S4,CPU,"[109,168]",22000,2,920,160,96,1000,423
4,GPU.S1,GPU,"[1,72]",120000,4,3000,8,96,1000,2310
5,GPU.S2,GPU,"[49,120]",140000,4,3000,8,96,1000,2695
6,GPU.S3,GPU,"[97,168]",160000,4,4200,8,96,1000,3080


In [2]:
df_datacenters = pd.read_csv('./data/datacenters.csv')
df_datacenters

,datacenter_id,cost_of_energy,latency_sensitivity,slots_capacity
0,DC1,0.25,low,25245
1,DC2,0.35,medium,15300
2,DC3,0.65,high,7020
3,DC4,0.75,high,8280


In [3]:

import numpy as np
import pandas as pd
from seeds import known_seeds
from utils import save_solution
from evaluation import get_actual_demand


def get_my_solution(d):
    # This is just a placeholder.
    return [{}]


seeds = known_seeds('training')

demand = pd.read_csv('./data/demand.csv')
for seed in seeds:
    # SET THE RANDOM SEED
    np.random.seed(seed)

    # GET THE DEMAND
    actual_demand = get_actual_demand(demand)

    # CALL YOUR APPROACH HERE
    solution = get_my_solution(actual_demand)

    # SAVE YOUR SOLUTION
    save_solution(solution, f'./output/{seed}.json')

    break



In [4]:

import uuid

def generate_random_id():
    return str(uuid.uuid4())

def fill_missing_timestep(df, min_time_step=1, max_time_step=168):
    full_range = pd.DataFrame({'time_step': range(min_time_step, max_time_step + 1)})
    df_filled = pd.merge(full_range, df, on='time_step', how='left')
    numeric_columns = ['high', 'low', 'medium']
    df_filled[numeric_columns] = df_filled[numeric_columns].fillna(0)
    df_filled['server_generation'] = df_filled['server_generation'].ffill()
    df_filled = df_filled.reset_index(drop=True)
    return df_filled

class Solution:

    def __init__(self, df_servers, df_data_centers, df_single_server=None, time_steps=[1, 196], verbose=False):
        self.df_servers = df_servers.copy()
        self.df_data_centers = df_data_centers.copy()
        self.datacenter_ids_to_index = {datacenter_id: i for i, datacenter_id in enumerate(df_data_centers['datacenter_id'].values)}
        self.time_steps = time_steps
        self.df_single_server = df_single_server
        if df_single_server is None:
            self.df_single_server = pd.read_csv("df_single_server_results.csv")

        self.verbose = verbose

    def add_buy_action(self, datacenter_id, server_generation, time_step):
        server_slot_size = self.df_servers.loc[self.df_servers['server_generation'] == server_generation, 'slots_size'].values[0]
        server_life_expectancy = self.df_servers.loc[self.df_servers['server_generation'] == server_generation, 'life_expectancy'].values[0]
        datacenter_slot_capacity = self.df_data_centers.loc[self.df_data_centers['datacenter_id'] == datacenter_id, 'slots_capacity'].values[0]
        
        datacenter_index = self.datacenter_ids_to_index[datacenter_id]
        self.data_center_slots[datacenter_index, time_step:time_step + server_life_expectancy] += server_slot_size
        if np.any(self.data_center_slots[datacenter_index, time_step:time_step + server_life_expectancy] > datacenter_slot_capacity):
            # if self.verbose:
            #     print(f"ERROR: Server {server_generation} cannot be added to data center {datacenter_id} at time step {time_step}. Server slots exceed data center capacity.")
            return False
        else:
            # get random server_generation
            server_id = generate_random_id()
            self.solution.append({
                "time_step": time_step,
                "action": "buy",
                "datacenter_id": datacenter_id,
                "server_generation": server_generation,
                "server_id": server_id
            })
            # if self.verbose:
            #     print(f"Server {server_id} added to data center {datacenter_id} at time step {time_step}")
            return True
        
    def search_buy_actions(self,):
            # server_generation = 'CPU.S1'
        # sensitivity = 'high'
        # datacenter_id = 'DC1'
        # utilization_threshold = 0.9
        

        for server_generation, sensitivity, datacenter_id, utilization_threshold \
            in self.df_single_server[['server_generation', 'latency_sensitivity', 'datacenter_id', 'average_U']].values:

            self.search_buy_action_for_server(server_generation=server_generation, 
                                            sensitivity=sensitivity,
                                            datacenter_id=datacenter_id,
                                            utilization_threshold=utilization_threshold)
        

    
    def search_buy_action_for_server(self, server_generation, sensitivity, datacenter_id, utilization_threshold):

        if self.verbose:
                print(f"Searching buy action for server generation {server_generation}, sensitivity {sensitivity}, datacenter {datacenter_id}, utilization threshold {utilization_threshold}")

        server_capacity = self.df_servers[self.df_servers['server_generation'] == server_generation]['capacity'].values[0]
        server_life_expectancy = self.df_servers[self.df_servers['server_generation'] == server_generation]['life_expectancy'].values[0]
        server_release_times = self.df_servers[self.df_servers['server_generation'] == server_generation]['release_time'].values[0]
        server_release_time_start, server_release_time_end = [int(num.strip()) for num in server_release_times.strip('[]').split(',')]

        demand_arr = self.actual_demand_by_server_generation[server_generation][sensitivity].copy().values
        total_buy_actions = 0
        for i in range(server_release_time_start, server_release_time_end + 1):
            demand_subarr = demand_arr[i - 1:i + server_life_expectancy - 1]
            while True:
                demand_subarr_capped = np.minimum(demand_subarr, server_capacity)
                average_utilization = np.mean(demand_subarr_capped/ server_capacity) 
                if average_utilization >= utilization_threshold:
                    success = self.add_buy_action(datacenter_id=datacenter_id, server_generation=server_generation, time_step=i)
                    if not success:
                        break
                    demand_arr[i - 1:i + server_life_expectancy - 1] -= demand_subarr_capped #* (1 - 0.07260491698699582)
                    total_buy_actions += 1
                else:
                    break
        self.actual_demand_by_server_generation[server_generation][sensitivity] = demand_arr
        if self.verbose:
            print(f"Bought {total_buy_actions} servers of generation {server_generation} for data center {datacenter_id} with sensitivity {sensitivity}")

    def solve(self, demand):
        self.solution = []
        self.df_demand = demand.copy()
        self.data_center_slots = np.zeros((self.df_data_centers.shape[0], self.time_steps[1]))


        ## query the demand for each server generation
        server_generation_unique = self.df_servers['server_generation'].unique()
        self.actual_demand_by_server_generation = {server_generation: demand[demand['server_generation'] == server_generation].sort_values('time_step') 
                                            for server_generation in server_generation_unique}
        for key in self.actual_demand_by_server_generation:
            self.actual_demand_by_server_generation[key] = fill_missing_timestep(self.actual_demand_by_server_generation[key])


        ## find buy actions
        self.search_buy_actions()


        return self.solution
        

        


# get solution

In [ ]:

import numpy as np
import pandas as pd
from seeds import known_seeds
from utils import save_solution
from evaluation import get_actual_demand

from utils import (load_problem_data,
                   load_solution)

def get_my_solution(d):
    _, df_datacenters, df_servers, df_selling_prices = load_problem_data()
    
    df_single_server = pd.read_csv("df_single_server_results.csv")
    # df_single_server = df_single_server[df_single_server['server_generation'] == 'CPU.S1']
    df_single_server = df_single_server[df_single_server['score'] > 0]
    solution = Solution(df_servers=df_servers, 
                        df_data_centers=df_datacenters, 
                        df_single_server=df_single_server,
                        verbose=True)
    solution.solve(d)
    return solution.solution


seeds = known_seeds('training')

demand = pd.read_csv('./data/demand.csv')
for seed in seeds:
    # SET THE RANDOM SEED
    np.random.seed(seed)

    # GET THE DEMAND
    actual_demand = get_actual_demand(demand)

    # CALL YOUR APPROACH HERE
    solution = get_my_solution(actual_demand)

    save_solution(solution, f'./output/{seed}.json')




In [ ]:
df_solution = pd.DataFrame(solution)

from utils import (load_problem_data,
                   load_solution)
from evaluation import evaluation_function


demand, datacenters, servers, selling_prices = load_problem_data()

total_score = 0
# LOAD SOLUTION
for seed in seeds:
    df_solution = load_solution(f'./output/{seed}.json')

    # EVALUATE THE SOLUTION
    score = evaluation_function(df_solution,
                                demand,
                                datacenters,
                                servers,
                                selling_prices,
                                seed=seed,
                                verbose=False)
    
    if score is not None:
        total_score += score

    print(f'Seed {seed}, Solution score: {score}')

print(f'Total score: {total_score}')

In [ ]:
# Seed 1741, Solution score: 609819645.2176845
# Seed 3163, Solution score: 748916183.9066746
# Seed 6053, Solution score: 807511356.6843554
# Seed 2237, Solution score: 842973353.3870907
# Seed 8237, Solution score: 687698394.303886
# Seed 8933, Solution score: 734086411.4131564
# Seed 4799, Solution score: 924013187.3786352
# Seed 1061, Solution score: 941214072.0697814
# Seed 2543, Solution score: 735743451.4769498
# Seed 8501, Solution score: 944582178.7449137
# Total score: 7976558234.583128
# First-position learderboard score: 8147659075.16152

# exp 1

In [ ]:

import numpy as np
import pandas as pd
from seeds import known_seeds
from utils import save_solution
from evaluation import get_actual_demand

from utils import (load_problem_data,
                   load_solution)

def get_my_solution(d, df_single_server=None):
    _, df_datacenters, df_servers, df_selling_prices = load_problem_data()
    
    if df_single_server is None:
        df_single_server = pd.read_csv("df_single_server_results.csv")
        # df_single_server = df_single_server[df_single_server['server_generation'] == 'CPU.S1']
        df_single_server = df_single_server[df_single_server['score'] > 0]
    solution = Solution(df_servers=df_servers, 
                        df_data_centers=df_datacenters, 
                        df_single_server=df_single_server,
                        verbose=False)
    solution.solve(d)
    return solution.solution

from utils import (load_problem_data,
                    load_solution)
from evaluation import evaluation_function


demand, datacenters, servers, selling_prices = load_problem_data()


df_single_server = pd.read_csv("df_single_server_results.csv")
df_single_server = df_single_server[df_single_server['server_generation'] == 'CPU.S1']
df_single_server = df_single_server[df_single_server['datacenter_id'] == 'DC3']
# df_single_server = df_single_server[df_single_server['score'] > 0]

for i in range(len(df_single_server)):
    seeds = known_seeds('training')

    demand = pd.read_csv('./data/demand.csv')
    for seed in seeds:
        # SET THE RANDOM SEED
        np.random.seed(seed)

        # GET THE DEMAND
        actual_demand = get_actual_demand(demand)

        # CALL YOUR APPROACH HERE
        solution = get_my_solution(actual_demand, df_single_server=df_single_server.iloc[:i+1])

        df_solution = pd.DataFrame(solution)
        
        # EVALUATE THE SOLUTION
        score = evaluation_function(df_solution,
                                    demand,
                                    datacenters,
                                    servers,
                                    selling_prices,
                                    seed=seed,
                                    verbose=False)

        print(f'i {i}, Solution score: {score}')

        break

